In [ ]:
# Week 2 project: Blueberry Yield Prediction
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error


In [ ]:
# Loading the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [3]:
train.head()
test.head()


id  clonesize  honeybee  bumbles  andrena  osmia  MaxOfUpperTRange  \
0  15289       25.0      0.25     0.25     0.25   0.25              86.0   
1  15290       12.5      0.25     0.25     0.75   0.63              94.6   
2  15291       12.5      0.25     0.25     0.63   0.63              86.0   
3  15292       25.0      0.50     0.38     0.38   0.63              86.0   
4  15293       37.5      0.75     0.25     0.25   0.25              94.6   

   MinOfUpperTRange  AverageOfUpperTRange  MaxOfLowerTRange  MinOfLowerTRange  \
0              52.0                  71.9              62.0              30.0   
1              57.2                  79.0              68.2              33.0   
2              52.0                  71.9              62.0              30.0   
3              52.0                  71.9              62.0              30.0   
4              57.2                  79.0              68.2              33.0   

   AverageOfLowerTRange  RainingDays  AverageRainingDays  fruitset  fruitmass  \
0                  50.8         24.0                0.39  0.399367   0.408088   
1                  55.9          1.0                0.10  0.488048   0.442866   
2                  50.8         16.0                0.26  0.583379   0.487057   
3                  50.8         16.0                0.26  0.433014   0.422847   
4                  55.9         24.0                0.39  0.360996   0.388860   

       seeds  
0  31.394569  
1  36.846956  
2  40.037644  
3  33.116091  
4  29.558019

In [ ]:
# Exploring the data
train.info()
train.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15289 entries, 0 to 15288
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    15289 non-null  int64  
 1   clonesize             15289 non-null  float64
 2   honeybee              15289 non-null  float64
 3   bumbles               15289 non-null  float64
 4   andrena               15289 non-null  float64
 5   osmia                 15289 non-null  float64
 6   MaxOfUpperTRange      15289 non-null  float64
 7   MinOfUpperTRange      15289 non-null  float64
 8   AverageOfUpperTRange  15289 non-null  float64
 9   MaxOfLowerTRange      15289 non-null  float64
 10  MinOfLowerTRange      15289 non-null  float64
 11  AverageOfLowerTRange  15289 non-null  float64
 12  RainingDays           15289 non-null  float64
 13  AverageRainingDays    15289 non-null  float64
 14  fruitset              15289 non-null  float64
 15  fruitmass          

id                      0
clonesize               0
honeybee                0
bumbles                 0
andrena                 0
osmia                   0
MaxOfUpperTRange        0
MinOfUpperTRange        0
AverageOfUpperTRange    0
MaxOfLowerTRange        0
MinOfLowerTRange        0
AverageOfLowerTRange    0
RainingDays             0
AverageRainingDays      0
fruitset                0
fruitmass               0
seeds                   0
yield                   0
dtype: int64

In [ ]:
# Checking for missing values
train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)


In [6]:
# Feature and target selection
X = train.drop(['id', 'yield'], axis=1)
y = train['yield']

X_test = test.drop(['id'], axis=1)


In [7]:
# Scaling the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)


In [8]:
# Splitting the data
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [9]:
models = {
    "Linear": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.01),
    "ElasticNet": ElasticNet(alpha=0.01, l1_ratio=0.5)
}


In [10]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    print(f"{name} MAE: {mae:.4f}")


Linear MAE: 370.4475
Ridge MAE: 370.7442


c:\Users\JUNE\Desktop\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.878e+09, tolerance: 2.195e+06
  model = cd_fast.enet_coordinate_descent(


Lasso MAE: 370.8239
ElasticNet MAE: 371.3827


c:\Users\JUNE\Desktop\.venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.225e+08, tolerance: 2.195e+06
  model = cd_fast.enet_coordinate_descent(


In [11]:
# Polynomial regression
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_scaled)
X_val_poly = poly.transform(X_val)

poly_model = LinearRegression()
poly_model.fit(X_poly, y)
y_poly_pred = poly_model.predict(X_val_poly)

mae_poly = mean_absolute_error(y_val, y_poly_pred)
print(f"Polynomial Regression MAE: {mae_poly:.4f}")


Polynomial Regression MAE: 363.8900


In [ ]:
# Selecting the best model and making predictions
best_model = Ridge(alpha=1.0)
best_model.fit(X_scaled, y)
test_preds = best_model.predict(X_test_scaled)


In [ ]:
submission = pd.DataFrame({
    'id': test['id'],
    'target': test_preds
})

submission.to_csv('submission.csv', index=False)
submission.head()


id       target
0  15289  4310.869359
1  15290  6103.339248
2  15291  7257.079313
3  15292  4981.632216
4  15293  3672.506679